# Recomendador Baseado em Similaridade de Usuários

In [3]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import sys
sys.path.append('../auxiliarScripts')
from dataset_reader import SpotifyPlaylistDataset
from scipy.sparse import csr_matrix, lil_matrix, save_npz, load_npz
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_rows', 100)

In [4]:
sparse_matrix = load_npz('../sparse/spotify_sparse400000.npz')
sparse_matrix

<1000000x2261616 sparse matrix of type '<class 'numpy.float64'>'
	with 26206816 stored elements in Compressed Sparse Row format>

### Modelo que Retorna Playlists mais Próximas

In [14]:
class ClosestPlaylists:
    def __init__(self, dataset, distance='cosine'):
        self.dataset = dataset
        self.distance = distance
    
    def recommend(self, playlists, index_playlist):
        #argsort = np.argsort(pairwise_distances(playlists[index_playlist],playlists, metric=self.distance))[0]
        argsort = np.argsort(cosine_similarity(playlists[index_playlist],playlists))[0][::-1]

        # pega a informação da playlist a ser recomendada
        formatted_playlist = self.dataset.read_playlist_formatted(index_playlist)
        # playlist mais proxima da playlist a ser recomendada
        closest_playlist = dataset.read_playlist_formatted(argsort[1])

        # retorna apenas músicas que não tem em comum
        #return closest_playlist
        #return closest_playlist[closest_playlist['track_uri'].isin(formatted_playlist['track_uri'])]
        return closest_playlist[~closest_playlist['track_uri'].isin(formatted_playlist['track_uri'])]

In [15]:
dataset = SpotifyPlaylistDataset()
model1 = ClosestPlaylists(dataset)
model1.recommend(sparse_matrix, 6003).sort_values(by='track_name')

Carregando dados da slice 0-999
Carregando dados da slice 6000-6999
Carregando dados da slice 16000-16999


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
22,22,Full Force,spotify:track:3Q3XLVUfQ0HOxUOtCUYMpG,spotify:artist:1iQS32l0NPcZeJKVrvayS2,Ain't My Type Of Hype,spotify:album:0pVHAjPdvdUNGhOGEhlVGP,288373,Smoove
4,4,Lionel Richie,spotify:track:2Wb9ejnmy27DUTUe9YF5Ew,spotify:artist:3gMaNLQm7D9MornNILzdSl,All Night Long (All Night) - Single Version,spotify:album:1W1b52vaEcE3wfWk4WQH6q,260426,The Definitive Collection
1,1,Michael Jackson,spotify:track:1OOtq8tRnDM8kG2gqUPjAj,spotify:artist:3fMbdgg4jU18AjLCKBhRSm,Beat It - Single Version,spotify:album:1C2h7mLntPSeVYciMRTF4a,258040,Thriller 25 Super Deluxe Edition
29,29,Cameo,spotify:track:0Aj8EagrPfDoOe5OlUdrLC,spotify:artist:545idYhdloaSlLGBY8E9u2,Candy,spotify:album:1W8kHQQC7DXg1inR7ZWqhA,339933,Word Up
35,35,Zapp,spotify:track:362zcsyXMLbL7PNLhOovvm,spotify:artist:396Kh0m4wGUvcMUULw71yi,Computer Love,spotify:album:3C9IDn2KBTZKRQ3Ldmm2Vr,283626,The New Zapp IV U
18,18,Bobby Brown,spotify:track:5KG4OVGxSrFmNHGZBezJJn,spotify:artist:62sPt3fswraiEPnKQpAbdE,Don't Be Cruel,spotify:album:44w4sVYJVQzhH2dN3IwyIK,408306,Don't Be Cruel
40,40,Lakeside,spotify:track:64JkfFIUe4gQTCXCQESF2X,spotify:artist:74lTWE4DqbFU3Vn8z4uH72,Fantastic Voyage - Extended,spotify:album:6qEgsBxOzMnrWyc4iOhGQp,369800,The Best of Lakeside
23,23,Pebbles,spotify:track:38bP8r9yx49g6pdQ90NH1X,spotify:artist:702AvACMk5mieXIYlC2Gxx,Girlfriend - Single Version,spotify:album:2zwv0LKTv8HDnYGfvFBmwj,256493,Greatest Hits: Pebbles
41,41,Cheryl Lynn,spotify:track:3eudp9ZxZAGaDBOuWGrW2D,spotify:artist:6UfoTQXaV3DuqtDVjZIxwZ,Got to Be Real - Single Version,spotify:album:059jmsqbxhu2n78LMS0H3P,223173,Cheryl Lynn (Expanded Edition)
3,3,Whitney Houston,spotify:track:5tdKaKLnC4SgtDZ6RlWeal,spotify:artist:6XpaIBNiVzIetEPCWDvAFP,How Will I Know,spotify:album:2MH37enG6IPvNK5QFLyKes,275533,Whitney Houston


In [134]:
dataset.read_playlist_formatted(6003).sort_values(by='track_name')

Carregando dados da slice 6000-6999


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
9,9,Michael Jackson,spotify:track:3bMveJpCPytU79V5A61Bb7,spotify:artist:3fMbdgg4jU18AjLCKBhRSm,Baby Be Mine,spotify:album:1C2h7mLntPSeVYciMRTF4a,260186,Thriller 25 Super Deluxe Edition
6,6,Soul II Soul,spotify:track:7vvRkLPIvfjjmCIqNxBuEZ,spotify:artist:2sIx6SmAMw9IBySG3Uj0jf,Back To Life (However Do You Want Me) - Accape...,spotify:album:1y3jN9LkTstdnogs1Xywni,228266,Keep On Movin'
13,13,Vaughan Mason and Crew,spotify:track:4x9Wm15FC2xCuSPgVl7X7i,spotify:artist:5MBBgzM082fkQbg75YyKBO,"Bounce, Rock, Skate, Roll",spotify:album:0j6BUfyDEFpuR54LTpO57K,329000,"Bounce, Rock, Skate, Roll"
8,8,Michael Jackson,spotify:track:46eu3SBuFCXWsPT39Yg3tJ,spotify:artist:3fMbdgg4jU18AjLCKBhRSm,Don't Stop 'Til You Get Enough - Single Version,spotify:album:2ZytN2cY4Zjrr9ukb2rqTP,365466,Off the Wall
2,2,Bobby Brown,spotify:track:0s6e7ZafqOAUBDoQYGmxrc,spotify:artist:62sPt3fswraiEPnKQpAbdE,Every Little Step,spotify:album:44w4sVYJVQzhH2dN3IwyIK,237240,Don't Be Cruel
7,7,Kool & The Gang,spotify:track:1OsCKwNZxph96EkNusILRy,spotify:artist:3VNITwohbvU5Wuy5PC6dsI,Get Down On It - Single Version,spotify:album:52ZOtET6twyVA0ZBlDG2RP,212906,The Very Best Of Kool & The Gang
10,10,Rick James,spotify:track:13v3siPyvy5TTEZYmGPPse,spotify:artist:0FrpdcVlJQqibaz5HfBUrL,Give It To Me Baby,spotify:album:2DBFUBBqJQvfXpodPi2WP5,249573,Street Songs
1,1,Lisa Lisa & Cult Jam,spotify:track:3sRCesHh424gmCji7Fz9Gm,spotify:artist:1vgjN6nIPNKiiQGE9PtzTT,I Wonder If I Take You Home,spotify:album:69c909n13KcSJPXxi6kPhG,404213,Playlist: The Very Best Of Lisa Lisa & Cult Jam
4,4,"Evelyn ""Champagne"" King",spotify:track:5IhBjmG7k7z9HVkvPl17N5,spotify:artist:4JCt4xrbbBB9blkKwNlcJ7,I'm In Love,spotify:album:5SZVTXdGx4zFBQqTxVXUcE,305853,I'm In Love
5,5,"Evelyn ""Champagne"" King",spotify:track:4qiTjzbhUr2BWg7ngmqm3V,spotify:artist:4JCt4xrbbBB9blkKwNlcJ7,If You Want My Lovin',spotify:album:4BlJ6291OlTOAKnnXJMIQI,259360,"The Essential Evelyn ""Champagne"" King"


In [ ]:
cosine_similarity(sparse_matrix)

In [20]:
dataset.read_playlist_formatted(100000)

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,ZAYN,spotify:track:3NdDpSvN911VPGivFlV5d0,spotify:artist:5ZsFI1h6hIdQRw2ti0hz81,I Don’t Wanna Live Forever (Fifty Shades Darke...,spotify:album:5MxXY7DbFMUiHFTPUabgJJ,245200,I Don’t Wanna Live Forever (Fifty Shades Darker)
1,1,Halsey,spotify:track:1uoL9GX45dTnmrWQ7Ii1Ft,spotify:artist:26VFTg2z8YR0cCuwLzESi2,Not Afraid Anymore,spotify:album:4HLA2Fsva7IJafJrhG5u52,226800,Not Afraid Anymore
2,2,Bryson Tiller,spotify:track:43PuMrRfbyyuz4QpZ3oAwN,spotify:artist:2EMAnMvWE2eb56ToJVfCWs,Exchange,spotify:album:6eZdwrhB97A3EYx9QppGfl,194613,T R A P S O U L
3,3,PARTYNEXTDOOR,spotify:track:1wZqJM5FGDEl3FjHDxDyQd,spotify:artist:2HPaUgqeutzr3jx5a9WyDV,Come and See Me (feat. Drake),spotify:album:2FXGUAESmG5l9YPrzWPvHI,235477,PARTYNEXTDOOR 3 (P3)
4,4,Bryson Tiller,spotify:track:3pXF1nA74528Edde4of9CC,spotify:artist:2EMAnMvWE2eb56ToJVfCWs,Don't,spotify:album:6eZdwrhB97A3EYx9QppGfl,198293,T R A P S O U L
5,5,J. Cole,spotify:track:6I6NX6tjGsxFAsIfGzY9lJ,spotify:artist:6l3HvQ5sa6mXTsMTB19rO5,Deja Vu,spotify:album:3CCnGldVQ90c26aFATC1PW,264746,4 Your Eyez Only
6,6,Joey Bada$$,spotify:track:4ntsD33Pgsgk8rUyZLLiUV,spotify:artist:2P5sC9cVZDToPxyomzF1UH,DEVASTATED,spotify:album:6swV0WUnPygRIMgEKn6Ige,207058,ALL-AMERIKKKAN BADA$$
7,7,Noah Cyrus,spotify:track:2BrzlUj1u1CtvaJDGIKpsP,spotify:artist:55fhWPvDiMpLnE4ZzNXZyW,Make Me (Cry),spotify:album:46yAYzRhSMPA44m0MzSr6g,242081,Make Me (Cry)
